In [26]:
import numpy as np
import math
import torch

## Utility functions

### storing ascii value of letters

In [27]:
import string

# ld contains the index of every letter in letters array
ld  =  {}
letters = [*string.ascii_lowercase]
for i, c in enumerate(letters):
    ld[c] = i
    

### function to find modular multiplicative inverse of matrix 

In [28]:

# finds modular inverse of matrix and raises valueError if it doesnt exist
def ModMatrixInverse(A,p):   
  
  def modInv(a,p):     
    for i in range(1,p):
      if (i*a)%p==1:
        return i
    raise ValueError(str(a)+" has no inverse mod "+str(p))

  # Returns matrix A with the ith row and jth column deleted
  def minor(A,i,j):    
    A = np.array(A)
    minor = np.zeros(shape=(len(A)-1,len(A)-1))
    p = 0
    for s in range(0,len(minor)):
      if p == i:
        p = p+1
      q = 0
      for t in range(0,len(minor)):
        if q == j:
          q = q+1
        minor[s][t] = A[p][q]
        q = q+1
      p = p + 1
    return minor
  
  n = len(A)
  A = np.matrix(A)
  adj = np.zeros(shape=(n,n))

  for i in range(0,n):
    for j in range(0,n):
      adj[i][j]=((-1)**(i+j)*int(round(np.linalg.det(minor(A,j,i))))) % p

  return (modInv(int(round(np.linalg.det(A))),p)*adj)%p



## Hill cipher

### Encryption algorithm

In [29]:
def encrypt( key, cipher ):

    # replace letters in key and cipher by their ascii equivalent 
    key = [ ld[c] for c in key if c in letters ]
    cipher = [ ld[c] for c in cipher if c in letters ]

    # rc is row count for how many rows will be there in key matrix (key_mat)
    rc = int( len(key) / len(cipher) )
    key_mat = np.array([[ 0 for _ in range(rc) ] for _ in range(rc)])

    # assigning values to key matrix
    for i, c in enumerate(key):
        key_mat[math.floor(i/rc)][i%rc] = np.int64(c)
    print(key_mat)

    # standard procedure of hill cipher - matrix multiply key_mat with cipher and mod by 26 to get the plaintext
    out_raw = torch.matmul(torch.tensor( cipher ), torch.tensor( np.int64(key_mat.transpose()) ) ) % 26

    # out_raw contains the ascii values. so here we are converting these values to characters 
    out = [ letters[int(i)] for i in out_raw ]

    encrypted = ''.join( out )

    return encrypted



### Decryption algorithm

In [30]:
def decrypt( key, cipher ):

    # replace letters in key and cipher by their ascii equivalent 
    key = [ ld[c] for c in key if c in letters ]
    cipher = [ ld[c] for c in cipher if c in letters ]

    # rc is row count for how many rows will be there in key matrix (key_mat)
    rc = int( len(key) / len(cipher) )
    key_mat = np.array([[ 0 for _ in range(rc) ] for _ in range(rc)])
 

    # assigning values to key matrix
    for i, c in enumerate(key):
        key_mat[math.floor(i/rc)][i%rc] = c

    try:
        key_mat = ModMatrixInverse( key_mat, 26 )       # find modular inverse of key matrix
    except ValueError:
        print( "inverse of key doesnt exist" )

    # then just standard procedure matrix multiply key_mat with cipher and mod by 26 to get the plaintext
    out_raw = np.matmul( np.array( cipher ), key_mat.transpose() ) % 26

    # out_raw contains the ascii values. so here we are converting these values to characters 
    out = [ letters[int(i)] for i in out_raw ]

    return ''.join(out)


In [31]:
# key = "AWESOME INTRODUCTION TO COMPUTER SECURITY AND FORENSICS".lower()
key = "AWESOME INTRODUCTION TO COMPUTER SECURITY AND FORENSICS".lower()
cipher = "SUST CSE".lower()

print( f"Encrypting \"{cipher}\" using key = {key}" )
encrypted = encrypt( key, cipher )
print( "Decrypted text: " + encrypted )

print( f"Decrypting \"{encrypted}\" using key = {key}" )
decrypted = decrypt( key, encrypted )


Encrypting "sust cse" using key = awesome introduction to computer security and forensics
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Decrypted text: wjctkzu
Decrypting "wjctkzu" using key = awesome introduction to computer security and forensics
inverse of key doesnt exist


I couldn't find a 7x7 invertible matrix with integers that are less than 26. But i did implement the decryption algorithm

The following cell is proof that decryption algorithm works

In [32]:
encrypted = "POH".lower()
key = "GYBNQKURP".lower()

print( f"Decrypting \"{encrypted}\" using key = {key}" )
print( "Decrypted text: " + decrypt(key, encrypted) )

Decrypting "poh" using key = gybnqkurp
Decrypted text: act
